In [23]:
from quartic_sdk import GraphqlClient
from pprint import pprint

class Client:

    def __init__(self, url, username, password):
        self._graphql_client = GraphqlClient(url=url,
            username=username, password=password)

    def graphql_client(self):
        return self._graphql_client
    
    def get_country(self):
        country_query = '''
           query MyQuery {
              Country {
                id
                name
              }
            }
        '''
        return self._graphql_client.execute_query(country_query)["data"]["Country"]
    
    def get_state(self):
        state_query = '''
           query MyQuery {
              State {
                id
                name
              }
            }
        '''
        return self._graphql_client.execute_query(state_query)['data']['State']
    
    def get_tag_measurements(self):
        tag_measurements_query = '''
           query MyQuery {
              TagMeasurements {
                id
                name
              }
            }
        '''
        return self._graphql_client.execute_query(tag_measurements_query)['data']['TagMeasurements']

In [24]:
#making a Client object; add your username and password in second and third arguement respectively  
client = Client("https://stag.quartic.ai", "dev123", "dev@quartic.ai")

2022-12-03 10:35:25 LAPTOP-BI53MUD2 asyncio[9000] DEBUG Using selector: SelectSelector


In [29]:

#add the mutation schema
site_mutation = lambda country, state, pinCode, addressLine1, name : f'''
    mutation MyMutation {{
    __typename
        SiteCreate(newSite: {{country: {country}, state: {state}, pinCode: "{pinCode}", addressLine1: "{addressLine1}", name: "{name}"}}) {{
            ok
            site {{
            id
            name
            }}
            errors {{
            messages
            field
            }}
        }}
    }}
'''

#define the function (Make sure to edit params as per the mutation)
def create_site(client, country, state, pinCode, addressLine1, name):
    mutation = site_mutation(country, state, pinCode, addressLine1, name)
    return client.graphql_client().execute_query(mutation)

#create the site
site = create_site(client, 10, 181, "123456", "test_address", "testSite")
print(site)
    

2022-12-03 12:08:57 LAPTOP-BI53MUD2 asyncio[9000] DEBUG Using selector: SelectSelector


{'data': {'__typename': 'MutationHolder', 'SiteCreate': {'ok': True, 'site': {'id': '31', 'name': 'testSite'}, 'errors': None}}}


In [34]:
#the documentation way of doing it

#define the function (Make sure to edit params as per the mutation)
def create_site2(client, country, state, pinCode, addressLine1, name):
    gql_mutation = """
    mutation MyMutation($country: ID!, $state: ID!, $pinCode: String!, $addressLine1: String! $name: String!) {
      SiteCreate(newSite: {country: $country, state: $state, pinCode: $pinCode, addressLine1: $addressLine1, name: $name }) {
        ok
        errors {
          field
          messages
        }
        site {
          id
          name
        }
      }
    }
    """
    
    return client.graphql_client().execute_query(gql_mutation,
        {"country": country, "state": state, "pinCode": pinCode, "addressLine1": addressLine1, "name": name})

#create the site
site2 = create_site2(client, 10, 181, "123456", "test_address", "testSite2")
print(site2)

2022-12-03 12:23:55 LAPTOP-BI53MUD2 asyncio[9000] DEBUG Using selector: SelectSelector


{'data': {'SiteCreate': {'ok': True, 'errors': None, 'site': {'id': '32', 'name': 'testSite2'}}}}


In [38]:
#update the site
def update_site(client, id, country, state, pinCode, addressLine1, name):
    gql_mutation = """
    mutation MyMutation($id: ID!, $country: ID!, $state: ID!, $pinCode: String!, $addressLine1: String! $name: String!) {
      SiteUpdate(updateSite: {id: $id, country: $country, state: $state, pinCode: $pinCode, addressLine1: $addressLine1, name: $name }) {
        ok
        errors {
          field
          messages
        }
        site {
          id
          name
        }
      }
    }
    """
    
    return client.graphql_client().execute_query(gql_mutation,
        {"id": id, "country": country, "state": state, "pinCode": pinCode, "addressLine1": addressLine1, "name": name})

#create the site
site3 = update_site(client, 28, 10, 181, "000456", "testupdate_address", "testSite2")
print(site3)

2022-12-03 15:02:08 LAPTOP-BI53MUD2 asyncio[9000] DEBUG Using selector: SelectSelector


{'data': {'SiteUpdate': {'ok': True, 'errors': None, 'site': {'id': '28', 'name': 'testSite2'}}}}
